In [1]:
from grand import lig_samplers as ls
from grand import lig_utils as lu

import openmm
import openmm.unit
import openff.toolkit
import pdbfixer

import numpy as np
import mdtraj as md
from scipy.spatial import Voronoi, Delaunay

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
datadir = '../../examples/lysozyme/'
# Load protein and ligand
prot = openmm.app.PDBFile(f'{datadir}/apo.pdb')
lig = openff.toolkit.Molecule.from_smiles("c1ccccc1O")
lig.generate_conformers(n_conformers=1000)
lig_top = lig.to_topology().to_openmm()
lig_pos = lig.conformers[0].to_openmm()

# Load forcefield
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml',f'{datadir}/phenol_openff.xml')

In [3]:
# Add one ligand to the system
sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, lig_top, lig_pos, n=1, output=f'{datadir}/ncmc.pdb')

100%|██████████| 1/1 [00:00<00:00, 40.65it/s]


In [4]:
# Load pdb with protein and ligand
pdb = openmm.app.PDBFile(f'{datadir}/ncmc.pdb')

In [5]:
# Solvate the system
#modeller = openmm.app.Modeller(pdb.topology, pdb.positions)
#modeller.addSolvent(forcefield=ff,padding=1*openmm.unit.nanometer)
#openmm.app.PDBFile.writeFile(modeller.topology, modeller.positions, f'{datadir}/ncmc_sol.pdb')

In [6]:
# Compute Voronoi vertices
traj = md.load(f'{datadir}/apo.pdb')
traj = traj.atom_slice(traj.top.select('name CA'))
vor = Voronoi(traj.xyz[0])
vor = vor.vertices[np.prod((vor.vertices > traj.xyz.min()) & (vor.vertices < traj.xyz.max()),axis=1).astype(bool)]

In [7]:
frag_res_ids = []
frag_atom_ids = []
alchemical_atom_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)
        for atom in residue.atoms():
            frag_atom_ids.append(atom.index)
    for atom in residue.atoms():
        alchemical_atom_ids.append(atom.index)

In [8]:
pdb_system = ff.createSystem(pdb.topology)

In [9]:
reporter = openmm.app.DCDReporter(f"{datadir}/ncmc_test.dcd", reportInterval=100)

In [10]:
# Initialize Sampler
sampler = ls.BaseNCMCSampler(pdb_system,pdb.topology,pdb.positions,reporters=[reporter],alchemical_atoms=alchemical_atom_ids, overwrite=True,
                             nsteps_neq=1000,nsteps_eq=100, frag_atoms=frag_atom_ids, insert_points_list=[openmm.Vec3(2.7880, 0.6749, 0.3701)])

In [11]:
# Run sampler
sampler.ncmc_move(5)

 20%|██        | 1/5 [00:01<00:06,  1.66s/it]

26010.99200963974 kJ/mol nan kJ/mol nan kJ/mol -1.005388885587886e+20 kJ/mol rejected


 40%|████      | 2/5 [00:03<00:04,  1.65s/it]

26050.199300944805 kJ/mol nan kJ/mol nan kJ/mol -1.0475283356640294e+20 kJ/mol rejected


 60%|██████    | 3/5 [00:04<00:03,  1.65s/it]

26004.272451400757 kJ/mol nan kJ/mol nan kJ/mol -1.1093742271483347e+20 kJ/mol rejected


 80%|████████  | 4/5 [00:06<00:01,  1.65s/it]

26040.18111819029 kJ/mol nan kJ/mol nan kJ/mol -1.0250179419324914e+20 kJ/mol rejected


100%|██████████| 5/5 [00:08<00:00,  1.76s/it]

26024.908638238907 kJ/mol nan kJ/mol nan kJ/mol -9.72848545746366e+19 kJ/mol rejected


In [12]:
# Print algorithm of integrator
sampler.insert_integrator.pretty_print()

step      0 : if(step = 0):
step      1 :    constrain positions
step      2 :    constrain velocities
step      3 :    protocol_work <- 0.0
step      4 :    lambda <- 0
step      5 :    protocol_work <- 0
step      6 :    step <- 0
step      7 :    lambda_step <- 0
step      8 :    lambda_sterics <- min(1.0, 2.0*lambda)
step      9 :    lambda_electrostatics <- max(0.0, 2.0*(lambda-0.5))
step     10 : end
step     11 : if(step >= 0):
step     12 :    if(step < n_steps_per_cycle):
step     13 :       allow forces to update the context state
step     14 :       if(has_kT_changed = 1):
step     15 :          sigma <- sqrt(kT/m)
step     16 :          has_kT_changed <- 0
step     17 :       end
step     18 :       old_pe <- energy
step     19 :       old_ke <- sum(0.5 * m * v * v)
step     20 :       x <- x + ((dt / 2) * v)
step     21 :       x1 <- x
step     22 :       constrain positions
step     23 :       v <- v + ((x - x1) / (dt / 2))
step     24 :       constrain velocities
step   